# A.1 - What is PyTorch?

## A.1.3 - Installing PyTorch

In [52]:
# Import PyTorch and check version
import torch
torch.__version__ # 2.2.2

'2.2.2'

In [53]:
# Check CUDA availability
torch.cuda.is_available() # False

False

In [54]:
# Check MPS availability (for Apple M1 chip)
print(torch.backends.mps.is_available()) # False

False


## Exercise A.1 - Page 257
Finished installing and setting up PyTorch on my computer.

## Exercise A.2 - Page 257
Finished running the supplementary code and everything outputed correctly. My environment is set up correctly.

# A.2 - Understanding tensors

## A.2.1 - Scalars, vectors, matrices, and tensors

In [55]:
# Initialize tensors with various dimensions
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2], [3, 4]])
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])


## A.2.2 - Tensor data types

In [56]:
# Tensor data types
tensor1d = torch.tensor([1, 2, 3])
print(tensor1d.dtype)

torch.int64


In [57]:
# Intialize float tensor
floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

torch.float32


In [58]:
# Convert to float data type
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

torch.float32


## A.2.3 - Common PyTorch tensor operations

In [59]:
# Initialize a tensor
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(tensor2d)

tensor([[1, 2, 3],
        [4, 5, 6]])


In [60]:
# Tensor shape - 2 rows and 3 columns
print(tensor2d.shape)

torch.Size([2, 3])


In [61]:
# Tensor reshaping
print(tensor2d.reshape(3, 2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [62]:
# Tensor views
print(tensor2d.view(3, 2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [63]:
# Tensor transposing
print(tensor2d.T)

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [64]:
# Tensor multiplication using matmul
print(tensor2d.matmul(tensor2d.T))

tensor([[14, 32],
        [32, 77]])


In [65]:
# Tensor multiplication using @
print(tensor2d @ tensor2d.T)

tensor([[14, 32],
        [32, 77]])


# A.3 - Seeing models as computation graphs

In [66]:
# Seeing models as computation graphs
import torch.nn.functional as F
y = torch.tensor([1.0])             # True label
x1 = torch.tensor([1.1])            # Input feature
w1 = torch.tensor([2.2])            # Weight parameter
b = torch.tensor([0.0])             # Bias unit
z = x1 * w1 + b                     # Net input
a = torch.sigmoid(z)                # Activation and output
loss = F.binary_cross_entropy(a, y)
print(f"The loss is {loss}.")

The loss is 0.0851878821849823.


# A.4 - Automatic differentiation made easy

In [67]:
# Computing gradients via autograd
from torch.autograd import grad
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

In [68]:
# Using grad function manually
z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [69]:
# Automated process - PyTorch compute the gradients of all the leaf nodes in this graph.
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


# A.5 - Implementing multilayer NN

In [70]:
# A multilayer perceptron with two hidden layers
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30), # 1st hidden layer
            torch.nn.ReLU(),
            torch.nn.Linear(30, 20), # 2nd hidden layer
            torch.nn.ReLU(),
            torch.nn.Linear(20, num_outputs) # output layer
        )
    
    def forward(self, x):
        return self.layers(x)

In [71]:
# Print model description
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [72]:
# Print the total number of trainable model parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable model parameters: {num_params}")

Total number of trainable model parameters: 2213


In [73]:
# Print the weight matrix - Random output each time
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0562, -0.0722, -0.0565,  ..., -0.0430,  0.0411, -0.0381],
        [ 0.0316, -0.1174, -0.0950,  ...,  0.0886, -0.0235,  0.1009],
        [ 0.0683,  0.0859, -0.0479,  ...,  0.0815,  0.1155,  0.0812],
        ...,
        [ 0.1331,  0.0627,  0.0924,  ..., -0.0744,  0.0756,  0.1294],
        [ 0.0401,  0.0788,  0.0775,  ..., -0.1309, -0.1004, -0.0429],
        [-0.0544, -0.0821, -0.0656,  ..., -0.1353, -0.0471, -0.1180]],
       requires_grad=True)


In [74]:
# Print the shape of the weight matrix
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [75]:
# Print the bias
print(model.layers[0].bias)

Parameter containing:
tensor([-0.0765,  0.0409, -0.0436,  0.0777,  0.1362, -0.0983, -0.0963,  0.0300,
        -0.1165, -0.1015, -0.0251,  0.0865, -0.1027, -0.0449,  0.0783,  0.0033,
        -0.1055,  0.1195, -0.0214,  0.0571, -0.0415,  0.1186,  0.0293,  0.0410,
         0.1317,  0.1336,  0.1372,  0.1015,  0.0797, -0.1064],
       requires_grad=True)


In [76]:
# Print the size for bias
print(model.layers[0].bias.shape)

torch.Size([30])


In [77]:
# Make the random number initialization reproducible
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [78]:
# A forward pass
torch.manual_seed(123)
X = torch.rand((1, 50))
output = model(X)
print(output)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [79]:
# For prediction after training
with torch.no_grad():
    output = model(X)
print(output)

tensor([[-0.1262,  0.1080, -0.1792]])


In [80]:
# Compute class-membership probabilities
with torch.no_grad():
    output = torch.softmax(model(X), dim=1)
print(output)

tensor([[0.3113, 0.3934, 0.2952]])


# A.6 - Setting up efficient data loaders

In [81]:
# Creating a small toy dataset
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0,0,0,1,1])
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0,1])

In [82]:
# Defining a custom Dataset class
from torch.utils.data import Dataset
class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        this_x = self.features[index]
        this_y = self.labels[index]
        return this_x, this_y
    
    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)
print(len(train_ds))

5


In [83]:
# Instantiating data loaders
from torch.utils.data import DataLoader
torch.manual_seed(123)

train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0 # The number of background processes
)

test_loader = DataLoader(
    dataset = test_ds,
    batch_size = 2,
    shuffle = False,
    num_workers = 0
)

In [84]:
# Print the training data from data loader
for idx, (x, y) in enumerate(train_loader):
    print(f'Batch {idx + 1}:', x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [85]:
# Drop the last batch
train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0, # The number of background processes
    drop_last = True
)

In [86]:
# Print the training data from data loader
for idx, (x, y) in enumerate(train_loader):
    print(f'Batch {idx + 1}:', x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


# A.7 - A typical training loop

In [87]:
# Neural network training in PyTorch
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Logging
        print(f"Epoch: {epoch+1:d}/{num_epochs:d} | Batch {batch_idx:d}/{len(train_loader):d} | Train Loss: {loss:.5f}")
    model.eval()

Epoch: 1/3 | Batch 0/2 | Train Loss: 0.74874
Epoch: 1/3 | Batch 1/2 | Train Loss: 0.64503
Epoch: 2/3 | Batch 0/2 | Train Loss: 0.44226
Epoch: 2/3 | Batch 1/2 | Train Loss: 0.12562
Epoch: 3/3 | Batch 0/2 | Train Loss: 0.02691
Epoch: 3/3 | Batch 1/2 | Train Loss: 0.00433


## Exercise A.3 - Page 275
- Input layer: 2 x 30 + 30 = 90
- Second layer: 30 x 20 + 20 = 620
- Output layer: 20 x 2 + 2 = 42
- Total: 90 + 620 + 42 = 752 parameters.

In [88]:
# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [89]:
# Obtain the class membership probabilities
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [90]:
# Convert class membership probabilities into class label predictions
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [91]:
# Convert class membership probabilities into class label predictions from outputs
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [92]:
# Check correctness for each instance
predictions == y_train

tensor([True, True, True, True, True])

In [93]:
# See how much are correct
torch.sum(predictions == y_train)

tensor(5)

In [94]:
# Generalize the computation of the prediction accuracy
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total = 0
    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total += len(compare)
    return (correct / total).item()

In [95]:
# Print training accuracy
print(compute_accuracy(model, train_loader))

1.0


In [96]:
# Print test accuracy
print(compute_accuracy(model, test_loader))

1.0


# A.8 - Saving and loading models

In [97]:
# Save models in PyTorch
torch.save(model.state_dict(), "model.pth")

In [98]:
# Retore the model from disk
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

# A.9 - Optimizing training performance with GPUs

## A.9.1 - PyTorch computations on GPU devices

In [99]:
# Check availability
print(torch.cuda.is_available())

False


In [100]:
# PyTorch computations on CPU
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


## A.9.2 - Single-GPU training

In [101]:
# Train on a single GPU
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Logging
        print(f"Epoch: {epoch+1:d}/{num_epochs:d} | Batch {batch_idx:d}/{len(train_loader):d} | Train Loss: {loss:.5f}")
    model.eval()

Epoch: 1/3 | Batch 0/2 | Train Loss: 0.74874
Epoch: 1/3 | Batch 1/2 | Train Loss: 0.64503
Epoch: 2/3 | Batch 0/2 | Train Loss: 0.44226
Epoch: 2/3 | Batch 1/2 | Train Loss: 0.12562
Epoch: 3/3 | Batch 0/2 | Train Loss: 0.02691
Epoch: 3/3 | Batch 1/2 | Train Loss: 0.00433


## Exercise A.4 - Page 282
- I don't have a GPU, so I couldn't finish this exercise.
- What I learned from Appendix C is that on V100, the computation is approximately four times faster.

In [102]:
# Time the matrix multiplication calculation on CPU
a = torch.rand(100, 200)
b = torch.rand(200, 300)
%timeit a@b

The slowest run took 9.29 times longer than the fastest. This could mean that an intermediate result is being cached.
1.74 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## A.9.3 - Training with multiple GPUs
- I cannot access multiple GPUs, so I skipped this section in the Appendix A.